# Entity Extraction from New Hampshire Case Law
*With IBM Granite Models*

The [New Hampshire Case Law Dataset](https://huggingface.co/datasets/free-law/nh) comes from the Caselaw Access Project via Hugging Face.

## In this notebook
This notebook contains instructions for performing entity extraction.

## Prerequisites

To get started, you'll need:
* A [Replicate account](https://replicate.com/) and API token.

## Setting up the environment

### Install dependencies

Granite Kitchen comes with a bundle of dependencies that are required for notebooks. See the list of packages in its [`setup.py`](https://github.com/ibm-granite-community/granite-kitchen/blob/main/setup.py). 

In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate \
    datasets \
    transformers \
    tiktoken

## Selecting System Components

### Choose your LLM
The LLM will be used for answering the question, given the retrieved text.

Follow the instructions in [Getting Started with Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/cee1513c77429d7ddbf0e5a49b29b7bc9ca0d996/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb), selecting a Granite Code model from the [`ibm-granite`](https://replicate.com/ibm-granite) org.

To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [ ]:
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import set_env_var

set_env_var("REPLICATE_API_TOKEN")

model = Replicate(
    model="ibm-granite/granite-3.0-8b-instruct",
)

## Get the tokenizer

Retrieve the tokenizer used by your chosen LLM.

In [ ]:
from transformers import AutoTokenizer

model_path = "ibm-granite/granite-3.0-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)

## Acquiring the Data

We will use a New Hampshire case law dataset to help the model answer questions about NH laws.

### Download the documents

Download the [New Hampshire CAP Caselaw](https://huggingface.co/datasets/free-law/nh) dataset from HuggingFace using the datasets library.

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader

# Load the documents from the dataset
loader = HuggingFaceDatasetLoader("free-law/nh", page_content_column="text")
documents = loader.load()
print("Document Count: " + str(len(documents)))

### Add metadata to the documents

Add the `source` field, which is used below, to the metadata.

In [ ]:
for doc in documents:
    doc.metadata['source'] = doc.metadata['id']

### Inspect the documents

In [ ]:
for doc in documents[:1]:
    print(doc.metadata, "\n")
    print(doc.page_content, "\n")

## Building the Document Database

We'll use the caselaw document database to retrieve the full text of the cases by case id.

### Create the database file and document table

In [ ]:
# # put the json objects in a sqlite database, keyed by id
# import sqlite3, os, json

# # remove database file if exists
# if os.path.isfile('data.db'):
#     os.remove('data.db')

# conn = sqlite3.connect('data.db')
# c = conn.cursor()

# # create the table if it doesn't exist. include id, text, and size
# c.execute('''CREATE TABLE IF NOT EXISTS data
#              (id INTEGER PRIMARY KEY UNIQUE,
#               metadata TEXT,
#               text TEXT,
#               char_count INTEGER)''')


### Insert the documents into the table

In [ ]:
# for doc in documents:
#     id = doc.metadata["id"]
#     c.execute("INSERT INTO data (id, metadata, text, char_count) VALUES (?,?,?,?)", (id, json.dumps(doc.metadata), doc.page_content, doc.metadata["char_count"]))
#     conn.commit()

### Count the documents

In [ ]:
# c.execute("SELECT count(*) FROM data")
# doc_count = c.fetchone()[0]
# print(f"Document count: {doc_count}")

## Extracting the entities

In this example, we take the caselaw text, split it into chunks, and extract entities from each chunk. 

### Split the document into chunks

Split the document into text segments that can fit into the model's context window.

In [ ]:
from langchain.text_splitter import TokenTextSplitter

# Split the documents into chunks
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=10)
chunks = text_splitter.split_documents(documents[:1])
print("Chunk Count: " + str(len(chunks)))

### Inspect the chunks

In [ ]:
import json
for i in range(1):
    print(chunks[i].page_content)
    print(json.dumps(chunks[i].metadata, indent=4))

# Provide taxonomy of entities

An LLM may produce this with the prompt:

```
I am building a knowledge graph from legal case law. What are the entities I should extract for this knowledge graph?
Prefix the major categories with numbers, and the minor categories with letters.
```

In [ ]:
query = """
Here are the categories of entity I want you to consider:

1. Case Information:
A. Case Name (e.g., "Brown v. Board of Education")
B. Docket Number (unique case identifier)
C. Court (e.g., Supreme Court, District Court)
D. Jurisdiction (state or federal jurisdiction, e.g., California, United States)
2. Legal Parties:
A. Plaintiff/Petitioner (the party initiating the case)
B. Defendant/Respondent (the party responding to the case)
C. Appellant/Appellee (for appeals)
3. Attorneys:
A. Counsel for Plaintiff/Petitioner (name, law firm, or organization)
B. Counsel for Defendant/Respondent (name, law firm, or organization)
4. Judges:
A. Judge(s)/Justice(s) (name and role: trial judge, appellate judge, presiding justice)
B. Panel Composition (for appellate cases, listing judges involved)
5. Legal Issues:
A. Legal Questions (the issues or questions of law presented to the court)
B. Claims (the specific claims or complaints raised by the plaintiff)
6. Legal Doctrines and Principles:
A. Statutes/Acts (e.g., "Civil Rights Act of 1964")
B. Precedents Cited (previous case law referred to)
C. Constitutional Provisions (e.g., "First Amendment," "Article III")
7. Facts and Context:
A. Material Facts (key facts on which the case is based)
B. Timeline (sequence of events leading up to and during the case)
8. Case Outcome:
A. Decision/Holding (the final judgment, such as "Affirmed," "Reversed")
B. Disposition (e.g., "dismissed with prejudice," "remanded")
C. Majority Opinion (summary or reasoning of the court's majority)
D. Concurring Opinion (opinion agreeing with the majority but for different reasons)
E. Dissenting Opinion (opinion disagreeing with the majority decision)
9. Procedural History:
A. Lower Court Rulings (previous decisions that led to the current case)
B. Appeals (sequence of appeals, appellate court involvement)
10. Relationships Between Entities:
A. Case Citation Relationship (e.g., "Case A cited Case B")
B. Statute Application (e.g., "Statute X was applied in Case Y")
C. Fact Relationships (linking individuals or events to legal consequences)
11. Court Documents:
A. Pleadings (complaints, answers, motions)
B. Briefs (e.g., appellate briefs, amicus briefs)
C. Orders (e.g., summary judgment, dismissal orders)
12. Dates:
A. Date of Filing (date the case was initiated)
B. Date of Decision (date the judgment was rendered)
C. Hearing Dates (important hearings or oral argument dates)
13. Legal Outcomes:
A. Remedies (e.g., "compensatory damages," "injunctive relief")
B. Sentences (in criminal cases, e.g., "5 years imprisonment")
14. Legal Concepts:
A. Standards of Review (e.g., "de novo," "abuse of discretion")
B. Burden of Proof (e.g., "preponderance of the evidence," "beyond a reasonable doubt")
"""

query = """
Case Name: The official name of the case (e.g., "Brown v. Board of Education").
Docket Number: The unique case identifier assigned to the case.
Court: The court where the case was heard (e.g., Supreme Court, District Court).
Jurisdiction: The jurisdiction under which the case falls (e.g., state or federal jurisdiction, like California, United States).
Plaintiff/Petitioner: The party initiating the case.
Defendant/Respondent: The party responding to the case.
Appellant/Appellee: The party appealing the decision and the party responding to the appeal, respectively.
Counsel for Plaintiff/Petitioner: The attorney or law firm representing the plaintiff/petitioner.
Counsel for Defendant/Respondent: The attorney or law firm representing the defendant/respondent.
Judge/Justice: The name of the judge or justice involved in the case, including their role (e.g., trial judge, appellate judge, presiding justice).
Panel Composition: The list of judges who heard the case (for appellate cases).
Legal Question: The issue or question of law presented to the court.
Claim: The specific claim or complaint raised by the plaintiff.
Statute/Act: The statute or act referenced or applied in the case (e.g., "Civil Rights Act of 1964").
Precedent Cited: Previous case law referred to in the case.
Constitutional Provision: The constitutional article or amendment referenced in the case (e.g., "First Amendment," "Article III").
Material Fact: The key fact on which the case is based.
Timeline: The sequence of events leading up to and during the case.
Decision/Holding: The final judgment of the court (e.g., "Affirmed," "Reversed").
Disposition: The outcome of the case (e.g., "dismissed with prejudice," "remanded").
Majority Opinion: The reasoning and summary of the decision given by the majority of the judges.
Concurring Opinion: An opinion that agrees with the majority's decision but for different reasons.
Dissenting Opinion: An opinion that disagrees with the majority decision.
Lower Court Ruling: A previous decision made by a lower court that led to the current case.
Appeal: The process and sequence of taking the case to a higher court.
Case Citation Relationship: A relationship indicating which case was cited by another case (e.g., "Case A cited Case B").
Statute Application: How a particular statute or law was applied in the case (e.g., "Statute X was applied in Case Y").
Fact Relationship: A relationship linking an individual or event to legal consequences within the case.
Pleading: A document such as a complaint, answer, or motion filed during the case.
Brief: A written document submitted by a party, including an appellate brief or amicus brief.
Order: A court order related to the case (e.g., summary judgment, dismissal order).
Date of Filing: The date when the case was initiated.
Date of Decision: The date on which the final decision was rendered.
Hearing Date: An important date for a hearing or oral argument in the case.
Remedy: Type of compensation or relief provided (e.g., "compensatory damages," "injunctive relief").
Sentence: In a criminal case, the sentence handed down (e.g., "5 years imprisonment").
Standard of Review: The standard used by the court in evaluating the case (e.g., "de novo," "abuse of discretion").
Burden of Proof: The level of proof required in the case (e.g., "preponderance of the evidence," "beyond a reasonable doubt").


Given this list of entity categories, find the entities of interest in the following text. List each entity, its relationship to the case, and short description of that relationship, in a format like this:
John A. Durkin; Defendant; The defendant in the case, against whom the plaintiff is seeking to invalidate an election.
Whether the Superior Court has jurisdiction to invalidate an election for United States Senator.; Legal Issue
State of New Hampshire; Jurisdiction; The state where the case is being heard and where the election took place.
New Hampshire Supreme Court; Court; The court where the case is being heard.
Louis C. Wyman; Plaintiff; The plaintiff in the case, seeking to invalidate an election for United States Senator.

\n{}"""


## Extract entities from each chunk of text

In [ ]:
responses = []
for chunk in chunks[:1]:
    print(f"Chunk of {chunk.metadata['id']}")
    response = model.invoke(query.format(chunk), max_tokens=1000)
    print(response)
    responses.append(response)

Using the extracted entities along with the text chunk, construct graph triples.

In [ ]:
query = """
List each entity named in these categories, along with a description of the entity:

{entities}
"""

response = model.invoke(query.format(entities=responses[0]))
print(response)